# IMPORTS

In [23]:
from collections import *
from functools import *
from itertools import *
import operator
import sys
import re
import math
import string
import bisect
import parse
import matplotlib.pyplot as plt
import numpy as np
import heapq
from matplotlib.path import Path
import ast
import hashlib
import unittest
import copy
from dataclasses import dataclass, field
import sympy
import networkx as nx
from pyvis.network import Network
from typing import *
from shapely.geometry import Polygon
from enum import StrEnum, Enum
from abc import ABC, abstractmethod

In [93]:
from enum import Enum
from collections import Counter
from typing import List, Generator, Tuple
from itertools import product

class Xmas(Enum):
    """
    Enum representing different symbols in the grid and their corresponding integer values.
    """
    M = 0  # Represents the "M" symbol.
    S = 1  # Represents the "S" symbol.
    A = 2  # Represents the "A" symbol.
    X = 3  # Represents the "X" symbol.

class XmasGrid:
    """
    A class to handle loading, processing, and analyzing a grid for specific patterns.
    """
    
    def __init__(self) -> None:
        """
        Initialize an empty grid with dimensions set to zero.
        """
        self.R: int = 0  # Number of rows in the grid
        self.C: int = 0  # Number of columns in the grid
        self.grid: List[str] = []  # The grid itself, stored as a list of strings
    
    def load(self, filename: str) -> None:
        """
        Load the grid from a file.

        Args:
            filename (str): The name of the file containing the grid.
        """
        with open(filename, "r") as f:
            self.grid = f.read().splitlines()
            self.R = len(self.grid)
            self.C = len(self.grid[0]) if self.R > 0 else 0
    
    def in_bounds(self, r: int, c: int) -> bool:
        """
        Check if a given coordinate is within the bounds of the grid.

        Args:
            r (int): Row index.
            c (int): Column index.

        Returns:
            bool: True if the coordinate is within bounds, False otherwise.
        """
        return 0 <= r < self.R and 0 <= c < self.C
    
    def diagonal_vectors(self, r: int, c: int) -> Generator[Tuple[int, int], None, None]:
        """
        Generate all diagonal vectors (dr, dc) for a given cell.

        Args:
            r (int): Row index.
            c (int): Column index.

        Yields:
            Tuple[int, int]: The change in row and column for diagonal neighbors.
        """
        for dr, dc in product(range(-1, 2), repeat=2):
            if abs(dr) + abs(dc) < 2:  # Skip non-diagonal and the origin
                continue
            yield dr, dc
    
    def diagonal_match(self, r: int, c: int) -> bool:
        """
        Check if the cell at (r, c) satisfies the diagonal match condition.

        Args:
            r (int): Row index.
            c (int): Column index.

        Returns:
            bool: True if the diagonal match condition is met, False otherwise.
        """
        if self.grid[r][c] != "A":
            return False
        
        diagonal_counters: Counter[int] = Counter()
        
        for dr, dc in self.diagonal_vectors(r, c):
            nr, nc = r + dr, c + dc
            if not self.in_bounds(nr, nc):
                return False
            symbol = self.grid[nr][nc]
            if symbol not in Xmas.__members__:
                return False
            diagonal_counters[abs(dr + dc)] += Xmas[symbol].value
        
        # Ensure all diagonal counters have a value of 1
        return all(v == 1 for v in diagonal_counters.values())
    
    def calculate(self, filename: str) -> None:
        """
        Process the grid and count the number of cells that satisfy the diagonal match condition.

        Args:
            filename (str): The name of the file containing the grid.
        """
        ans: int = 0
        self.load(filename)
        for r, c in product(range(self.R), range(self.C)):
            ans += self.diagonal_match(r, c)
        print(ans)

# Example Usage
xmas_grid = XmasGrid()
xmas_grid.calculate("big.txt")

1905


In [ ]:
from enum import Enum

class Symbol(Enum):
    X = 10
    M = 20
print(Symbol["X"])

10


In [ ]:
from collections import Counter
import string
def calculate(filename: str):
    with open(filename, "r") as f:
        data = f.read().splitlines()
        ans = 0
        pattern = r"\[([a-z]+)\]"
        p2 = r"\d+"
        for row in data:
            match = re.search(pattern, row)
            chars = match.groups()[0]
            match = re.search(p2, row)
            letters = [ch for ch in row[:match.start()] if ch in string.ascii_lowercase]
            counts = list(Counter(letters).items())
            counts.sort(key = lambda x: (-x[1], x[0]))
            arr = [ch for ch, _ in counts]
            ordered_string = "".join(arr[:5])
            val = int(match.group())
            if chars == ordered_string:
                ans += val
    print(ans)
calculate("small.txt")

0


In [ ]:
from collections import Counter
import string
def calculate(filename: str):
    with open(filename, "r") as f:
        data = f.read().splitlines()
        ans = 0
        pattern = r"\[([a-z]+)\]"
        p2 = r"\d+"
        for row in data:
            match = re.search(pattern, row)
            chars = match.groups()[0]
            match = re.search(p2, row)
            letters = [ch for ch in row[:match.start()] if ch in string.ascii_lowercase]
            counts = list(Counter(letters).items())
            counts.sort(key = lambda x: (-x[1], x[0]))
            arr = [ch for ch, _ in counts]
            ordered_string = "".join(arr[:5])
            val = int(match.group())
            if chars == ordered_string:
                ans += val
    print(ans)
calculate("small.txt")